In [9]:
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten, LeakyReLU
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import StratifiedKFold

import tensorflow as tf
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf


### 이미지 전처리

In [10]:
IMAGE_SIZE = (300, 300)

IMAGE_PATH = "./plant-pathology-2020-fgvc7/images/"
TEST_PATH = "./plant-pathology-2020-fgvc7/test.csv"
TRAIN_PATH = "./plant-pathology-2020-fgvc7/aug_train.csv"
#SUB_PATH = "./plant-pathology-2020-fgvc7/sample_submission.csv"

test_data = pd.read_csv(TEST_PATH)
train_data = pd.read_csv(TRAIN_PATH)

def load_image(image_id):
    file_path = image_id + ".jpg"
    image = cv2.imread(IMAGE_PATH + file_path)

    # 이미지를 224, 224로 줄이기
    image = cv2.resize(image, IMAGE_SIZE)

    return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

images = train_data["image_id"].apply(load_image)


In [11]:
images = np.stack(images)
labels = train_data[["healthy", "multiple_diseases", "rust", "scab"]].values

In [18]:
BATCH_SIZE = 4
LEARNING_RATE = 0.00025
K_NUM= 2

from tensorflow.keras.applications.efficientnet_v2 import EfficientNetV2B3
efmodel = EfficientNetV2B3(include_top=False, weights='imagenet', input_shape= (IMAGE_SIZE[1], IMAGE_SIZE[0], 3))
model = Sequential()
model.add(efmodel)
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(64, activation='leaky_relu'))
model.add(Dense(4, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE), metrics=['accuracy'])

In [19]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetv2-b3 (Function  (None, 10, 10, 1536)     12930622  
 al)                                                             
                                                                 
 flatten_3 (Flatten)         (None, 153600)            0         
                                                                 
 dropout_3 (Dropout)         (None, 153600)            0         
                                                                 
 dense_6 (Dense)             (None, 64)                9830464   
                                                                 
 dense_7 (Dense)             (None, 4)                 260       
                                                                 
Total params: 22,761,346
Trainable params: 22,652,130
Non-trainable params: 109,216
____________________________________

In [20]:
y_categorical = np.argmax(labels, axis=1)
k = 5  # Number of folds
stratified_kfold = StratifiedKFold(n_splits=k)

history = None
for i, (train_indices, test_indices) in enumerate(stratified_kfold.split(images, y_categorical)):
    if i != K_NUM:
        continue
        
    train_images, val_images = images[train_indices], images[test_indices]
    train_labels, val_labels = labels[train_indices], labels[test_indices]

    train_datagen = ImageDataGenerator(
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   rotation_range=5,
                                   shear_range=0.2,
                                   zoom_range=[0.9, 1.1],
                                   #brightness_range= [0.7, 1],
                                   fill_mode='nearest'
                                   )

    val_datagen = ImageDataGenerator()

    train_generator = train_datagen.flow(x=train_images, y=train_labels, batch_size=BATCH_SIZE, shuffle=True)
    val_generator = train_datagen.flow(x=val_images, y=val_labels, batch_size=BATCH_SIZE)
    
    modelpath = f'./model/train12/kfold{i}/epoch100.hdf5'
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5)

    history = model.fit(train_generator, epochs=100, verbose=1, callbacks=[reduce_lr], validation_data=val_generator)
    model.save(modelpath)

Epoch 1/100
365/365 [==============================] - 37s 78ms/step - loss: 0.9064 - accuracy: 0.7783 - val_loss: 0.3254 - val_accuracy: 0.9011 - lr: 2.5000e-04
Epoch 2/100
365/365 [==============================] - 28s 77ms/step - loss: 0.4700 - accuracy: 0.8586 - val_loss: 0.3257 - val_accuracy: 0.9011 - lr: 2.5000e-04
Epoch 3/100
365/365 [==============================] - 28s 78ms/step - loss: 0.5084 - accuracy: 0.8682 - val_loss: 0.3552 - val_accuracy: 0.9121 - lr: 2.5000e-04
Epoch 4/100
365/365 [==============================] - 29s 79ms/step - loss: 0.4186 - accuracy: 0.8943 - val_loss: 0.7809 - val_accuracy: 0.9148 - lr: 2.5000e-04
Epoch 5/100
365/365 [==============================] - 29s 79ms/step - loss: 0.4950 - accuracy: 0.8813 - val_loss: 0.3809 - val_accuracy: 0.9176 - lr: 2.5000e-04
Epoch 6/100
365/365 [==============================] - 29s 79ms/step - loss: 0.4092 - accuracy: 0.9019 - val_loss: 0.2223 - val_accuracy: 0.9313 - lr: 2.5000e-04
Epoch 7/100
365/365 [=======